In [2]:
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate

using SparseArrays

struct DiagonalIndices{Tv, Ti <: Integer}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::Vector{Ti}

    function DiagonalIndices{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti}
        # Check square?
        diag = Vector{Ti}(undef, A.n)

        for col = 1 : A.n
            r1 = Int(A.colptr[col])
            r2 = Int(A.colptr[col + 1] - 1)
            r1 = searchsortedfirst(A.rowval, col, r1, r2, Base.Order.Forward)
            if r1 > r2 || A.rowval[r1] != col || iszero(A.nzval[r1])
                throw(LinearAlgebra.SingularException(col))
            end
            diag[col] = r1
        end 

        new(A, diag) #
    end
end

DiagonalIndices(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti} = DiagonalIndices{Tv,Ti}(A)
#밖에서도 함수로 쓸수있게 따로 빼준건가?

#DiagonalIndices 특징은 Sparse matrix의 nz diagonal element vector로 변환시키며,
#diagonal term에 0이 있으면 안된다.
#원 matrix을 담고 있는 구조체임.
# A.nz는 col 순서대로 적어내린 values
# A.row는 nz 기준 해당 row index
# A.cor은 i에  해당하는 nz index 범위를 찍어줌

@inline getindex(d::DiagonalIndices, i::Int) = d.diag[i]
#get index를 선언해줘야 forwardsub!가 돌아갔음..



struct FastLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct FastUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct OffDiagonal{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end


"""
Forward substitution for the FastLowerTriangular type
"""
function forward_sub!(F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix
    @inbounds for col = 1 : A.n
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # ok
        for i = idx + 1 : (A.colptr[col + 1] - 1) #colptr인데 lower triangular이기 때문에 해당 col의 diagonal 아래 개수가나옴.
            x[A.rowval[i]] -= A.nzval[i] * x[col] # 이 term으로 x[n] 계산할때 그이전텀들이 다 마이너스 되어서 있음. 
        end
    end
    x
end

# solve  A(x- β × y) = α × b (여기서 b = x)
function forward_sub!(α, F::FastLowerTriangular, x::AbstractVector, β, y::AbstractVector)
    A = F.matrix

    @inbounds for col = 1 : A.n

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] = α * x[col] / A.nzval[idx] + β * y[col]

        # Substitute next values involving x[col]
        for i = idx + 1 : (A.colptr[col + 1] - 1)
            x[A.rowval[i]] -= A.nzval[i] * x[col]
        end
    end

    x
end

#(α×A)+(β×y)  
function mul!(α::T, O::OffDiagonal, x::AbstractVector, β::T, y::AbstractVector) where {T}
    # Specialize for β = 0 and β = 1
    A = O.matrix

    if β != one(T)
        if iszero(β)
            fill!(y, zero(T))
        else
            lmul!(β, y) #그냥 베타 곱해주는 함수
        end
    end

    @inbounds for col = 1 : A.n
        αx = α * x[col]
        diag_index = O.diag[col]
        for j = A.colptr[col] : diag_index - 1 #off diagonal 이기때문에 해당 제외 데이터 개수
            y[A.rowval[j]] += A.nzval[j] * αx #y를 솔루션이라고 하면 해당하는 
        end
        #off diagonal 하나 빼는게 힘들어서 이지랄/..
        for j = diag_index + 1 : A.colptr[col + 1] - 1
            y[A.rowval[j]] += A.nzval[j] * αx
        end
    end

    y
end

# (α×U×X)+(β×y)
function gauss_seidel_multiply!(α, U::StrictlyUpperTriangular, x::AbstractVector, β, y::AbstractVector, z::AbstractVector)
    A = U.matrix

    for col = 1 : A.n
        αx = α * x[col]
        diag_index = U.diag[col]
        @inbounds for j = A.colptr[col] : diag_index - 1
            z[A.rowval[j]] += A.nzval[j] * αx
        end
        z[col] = β * y[col] 
        #위에서 j = A.colptr[col] : diag_index - 1 만 건드리기때문에 β 곱하는게 우선시 됨.
    end
    z
end


gauss_seidel_multiply! (generic function with 1 method)

In [ ]:
#Init val
w = 0.5 # not greater than 2?
A_ = copy(A)
D = copy(Diagonal(A))
max_itter = 100
#Diagonal(A_).= 1/w *Diagonal(A_)
#M_sor = LowerTriangular(A_)

#Calc M & N
M_sor = (1/w-1) * D +  LowerTriangular(A)
M_ssor = w/(2-w) * M_sor * D^(-1) * Transpose(M_sor)
N = M_ssor - A


In [7]:

A_temp = LowerTriangular(A)
#A_temp = UpperTriangular(A)
A_temp = sparse(A_temp)
D_a = DiagonalIndices(A_temp)
LT = FastLowerTriangular(D_a.matrix,D_a)
b = [1.1 ;2.0 ;3.0 ;4.0]
display(Matrix(A_temp))
display(b)

UndefVarError: UndefVarError: LowerTriangular not defined

# Miscellaneous

In [4]:

using SparseArrays
using LinearAlgebra

 
A = sparse([1.0 2 3 0; 1 1 0 0;0 1 4 1; 1 0 0 10])
D = DiagonalIndices(A)

display("matrix A")
display(Matrix(A))
display("matrix A diag")
display(D.diag)
b = [1.1 ;2.0 ;3.0 ;4.0]

"""
A_temp = LowerTriangular(A)
#A_temp = UpperTriangular(A)
A_temp = sparse(A_temp)
D_a = DiagonalIndices(A_temp)
LT = FastLowerTriangular(D_a.matrix,D_a)
display(Matrix(A_temp))
"""
display(b)


"matrix A"

4×4 Array{Float64,2}:
 1.0  2.0  3.0   0.0
 1.0  1.0  0.0   0.0
 0.0  1.0  4.0   1.0
 1.0  0.0  0.0  10.0

"matrix A diag"

4-element Array{Int64,1}:
  1
  5
  8
 10

Base.Multimedia.display

In [29]:

function m_sor!(A, D::DiagonalIndices, w)
    for d_idx in D.diag 
        A.nzval[d_idx]  *= (1/w)
    end
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_sor!(F::FastLowerTriangular, A::SparseMatrixCSC,
                        x::AbstractVector, b::AbstractVector, max_itter)
    r =zeros(A.n)
    r[:] = b - A * x
    for i = 1 : max_itter 
        if norm( r, 2) < 10^-8 
            return x
        end
        x[:] += forward_sub!(F, r)
        r[:] = b - A * x
    end
    x
end




itter_sor! (generic function with 1 method)

In [47]:
#A = sparse([1.0 2 3 0; 1 1 0 0;0 1 4 1; 1 0 0 10])
#b = [1;2;3;4]
sol_1 = A\b
print(sol_1)

x = zeros(A.n)

w = 0.5 
m_sor = copy(A)
m_sor!(m_sor, D, w)
D_ms= DiagonalIndices(A)

x = itter_sor!(FastLowerTriangular(m_sor ,D_ms), A, x , b, 1000)


[-0.21017728652155654, -0.5480699350194204, 0.264100394433014, -0.160085511521196, -0.575080524235595, -0.7919143059121234, -0.18078843526160096, -0.6264961520165465, -0.4541512684928235, -0.36643642934680876, -0.4188770560308934, 0.10067663339435785, -0.152052033980938, -0.09015303115391067, -0.7698714524161533, -1.2839604445486528, -0.45260612538453215, -0.5163397304411811, -0.5866070180239953, -0.3306009462247115, -0.6833379252941238, -0.19213691072543712, -0.1488797643399878, -0.3604244202031451, -0.9598856624346039, -1.3271902203384913, -0.9207152022892243, -0.826277386050504, -0.42090240650435645, -0.4014089549054248, -0.53836529464317, -0.49567710751998384, -0.4329281132829362, -0.4081495244179296, -1.391035890454468, -1.6133670773422806, -1.2787317542698329, -1.4057431368530788, -0.1971833879825893, -0.284613324944497, -0.7727880055418137, -0.5425516879669174, -0.0368396173532256, -0.7061334812078042, -1.388893367459475, -1.1210602292963603, -0.8138953915652448, -0.644459501152

100-element Array{Float64,1}:
 -0.21017728509808375
 -0.5480699325104863 
  0.26410039742866837
 -0.16008550821528222
 -0.5750805207555416 
 -0.7919143025681876 
 -0.18078843231281996
 -0.626496149666229  
 -0.454151266879815  
 -0.3664364285403613 
 -0.4188770537486328 
  0.10067663753276102
 -0.15205202856413633
  ⋮                  
 -0.08545116104574216
  0.32116860686578935
 -0.24714137414245427
 -0.22818484954955479
 -0.3463157005952767 
 -0.02470667936788577
 -0.09217274132853995
 -0.4098966027020355 
 -0.6365760569620468 
 -0.633529309176767  
 -0.41086624178099485
 -0.12419948936052219

In [48]:
x

100-element Array{Float64,1}:
 -0.21017728509808375
 -0.5480699325104863 
  0.26410039742866837
 -0.16008550821528222
 -0.5750805207555416 
 -0.7919143025681876 
 -0.18078843231281996
 -0.626496149666229  
 -0.454151266879815  
 -0.3664364285403613 
 -0.4188770537486328 
  0.10067663753276102
 -0.15205202856413633
  ⋮                  
 -0.08545116104574216
  0.32116860686578935
 -0.24714137414245427
 -0.22818484954955479
 -0.3463157005952767 
 -0.02470667936788577
 -0.09217274132853995
 -0.4098966027020355 
 -0.6365760569620468 
 -0.633529309176767  
 -0.41086624178099485
 -0.12419948936052219

In [49]:
sor(A, b, 0.75, maxiter=10000)

100-element Array{Float64,1}:
 -0.21017728652155643 
 -0.5480699350194201  
  0.26410039443301436 
 -0.16008551152119538 
 -0.5750805242355941  
 -0.7919143059121225  
 -0.1807884352616003  
 -0.6264961520165462  
 -0.4541512684928233  
 -0.36643642934680865 
 -0.41887705603089326 
  0.10067663339435831 
 -0.15205203398093728 
  ⋮                   
 -0.08545116326876082 
  0.3211686057542981  
 -0.247141374949586   
 -0.22818485103558186 
 -0.3463157025881486  
 -0.024706681668987956
 -0.09217274373064724 
 -0.40989660500646613 
 -0.6365760589937295  
 -0.6335293107962168  
 -0.4108662428925116  
 -0.12419948991627419 

In [46]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 10
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
Afull = convert(Matrix, A)
@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)
b = randn(n^2)

typeof(A) = SparseMatrixCSC{Float64,Int64}
typeof(Afull) = Array{Float64,2}


100-element Array{Float64,1}:
  0.12623784496408763
 -2.3468794813834966 
  1.9166090582536102 
 -0.23920888512829233
 -0.5784508270929073 
 -1.1278278196026448 
  1.1478628422667982 
 -1.354705173870581  
 -0.23706547458394342
 -0.6809935026697    
 -0.8826696457022515 
  1.7138424693341203 
 -0.7339523682572253 
  ⋮                  
  0.10659685478060812
  1.2264526310840413 
 -0.1420101496925546 
  0.25421803664096554
 -0.4357528908303444 
  1.0522681632560082 
  1.0163143379698492 
 -0.09510235600663826
 -0.10653041583504672
 -0.9015114432463756 
 -0.8002850075887946 
 -0.4071003225268833 

In [16]:
w

0.5

In [24]:
Matrix(m_sor)

4×4 Array{Float64,2}:
 2.0  0.0  0.0   0.0
 1.0  2.0  0.0   0.0
 0.0  1.0  8.0   0.0
 1.0  0.0  0.0  20.0

In [8]:
sol_1

4-element Array{Float64,1}:
  19.7142857142857  
 -17.7142857142857  
   5.571428571428566
  -1.57142857142857 

In [53]:
Matrix(m_sor)

4×4 Array{Float64,2}:
 2.0  2.0  3.0   0.0
 1.0  2.0  0.0   0.0
 0.0  1.0  8.0   1.0
 1.0  0.0  0.0  20.0

In [39]:
A\b

4-element Array{Float64,1}:
 -0.7979274611398964 
  1.3989637305699483 
 -0.06735751295336795
  0.13989637305699482

In [30]:
Matrix(A)

4×4 Array{Float64,2}:
 2.0  2.0  3.0   0.0
 1.0  2.0  0.0   0.0
 0.0  1.0  8.0   1.0
 1.0  0.0  0.0  20.0

In [31]:
Matrix(m_sor)

4×4 Array{Float64,2}:
 1.0  2.0  3.0   0.0
 1.0  1.0  0.0   0.0
 0.0  1.0  4.0   1.0
 1.0  0.0  0.0  10.0